In [14]:
from tkinter import *
from tkinter import font as tkFont  # for convenience
import spotify_api
import pprint
import json
import numpy as np
import pandas as pd
import joblib
from numpy import array

nn_model = joblib.load('finalized_model.joblib')

root = Tk()
root.title('Codemy.com')
root.geometry("1200x800")

# Maybe you need to change the image name here
bg = PhotoImage(file = "ezgif.com-gif-maker.gif",master = root)

my_canvas = Canvas(root,width=1200,height = 800)
my_canvas.pack(fill="both",expand = True)
my_canvas.create_image(0,0,image = bg, anchor="nw")

# Add Label
my_canvas.create_text(630, 120, text = "Song Popularity",fill="white",font = "Times 100 italic bold")
my_canvas.create_text(450, 300, text = "Song  Name: ",font=('Arial','30','bold'))
my_canvas.create_text(630, 400, text = "__________________",font=('Arial','30','bold'))
my_canvas.create_text(425, 600, text = "Popularity: ",fill = "#5e9d54",font=('Arial','30','bold'))
my_canvas.create_text(725, 600, text = "__________________",fill = "#5e9d54",font=('Arial','30','bold'))

    
# Input Entry Bar
entry = Entry(my_canvas,bg = "#5e9d54",width=20, font = ('Arial','20','bold'), bd = 0, justify='center')
entry_window = my_canvas.create_window(630,390,window=entry)


# Output Entry Bar
output = Entry(my_canvas,bg = "#222222",fg = "#5e9d54",width=12, font = ('Arial','30','bold'), bd = 0, justify='center')
output_window = my_canvas.create_window(725, 590,window=output)
 
# The function, which we should change to connect API and our model
def setTextInput():
    text = entry.get()
    

    # create your own client_id and client_secret tokens and plug in variables as strings
    # https://cran.r-project.org/web/packages/spotidy/vignettes/Connecting-with-the-Spotify-API.html
    client_id = ""
    client_secret = ""

    spotify = spotify_api.SpotifyAPI(client_id, client_secret)
    track = spotify.search(text, search_type = "track", limit = 1)
    track_items = track["tracks"]["items"]
    track_id = track_items[0]["id"]

    #print(track_id)

    analysis_result = spotify.features(_id = track_id)
    #danceability  instrumentalness  liveness  loudness  speechiness
    danceability = analysis_result["danceability"]
    instrumentalness = analysis_result["instrumentalness"]
    liveness = analysis_result["liveness"]
    loudness = analysis_result["loudness"]
    speechiness = analysis_result["speechiness"]
    valence = analysis_result["valence"]
    
    danceability_min, danceability_max = 0.05690, 0.989
    instrumentalness_min, instrumentalness_max = 0.0, 0.999
    liveness_min, liveness_max = 0.00967, 1.0
    loudness_min, loudness_max = -52.45700, 3.744
    speechiness_min, speechiness_max = 0.02220, 0.967
    valence_min, valence_max = 0.01240, 1.0

    danceability = (danceability - danceability_min)/(danceability_max - danceability_min)
    instrumentalness = (instrumentalness - instrumentalness_min)/(instrumentalness_max - instrumentalness_min)
    liveness = (liveness - liveness_min)/(liveness_max - liveness_min)
    loudness = (loudness - loudness_min)/(loudness_max - loudness_min)
    speechiness = (speechiness - speechiness_min)/(speechiness_max - speechiness_min)
    valence = (valence - valence_min)/(valence_max - valence_min)

    #print(danceability)
    #print(instrumentalness)
    #print(liveness)
    #print(loudness)
    #print(speechiness)
    #print(valence)

    data = {'danceability':[danceability], 'instrumentalness':[instrumentalness], 'liveness':[liveness], 'loudness':[loudness], 'speechiness':[speechiness], 'valence':[valence]}
    df = pd.DataFrame(data)

    y_pred = nn_model.predict(df)
    #print(y_pred)
    result = y_pred[0]
    result_str = str(result)
    if result == 3:
        result_str = "3 : High"
    if result == 2:
        result_str = "2 : Medium"
    if result == 1:
        result_str = "1 : Low"
    output.delete(0,"end")
    output.insert(0, result_str)
    
# Add buttons    
go_button = Button(root, text = "GO",width=8, height=1,fg = "#fff", command=lambda:setTextInput()) #When hit the button, run setTextInput(). We can change this function to API. 
helv36 = tkFont.Font(family='Arial',weight='bold')
go_button['font'] = helv36

go_button.configure(bg = "#c78a86")
button_window = my_canvas.create_window(850, 300, window=go_button)


root.mainloop()